In [ ]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.tools import tool

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "너는 뛰어나고 유능한 주문봇이야"
            "이전 대화와 현재 고객이 입력한 메시지 모두를 꼼꼼히 파악하여 답변해줘."
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),

    ]
)

model = ChatOpenAI()

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

@tool
def get_recent_orders_by_status(user, status):
    """
    Retrieve the three most recent orders with a specific status for the given user.
    """
    # status_changes 를 통해 필터링하고, 주문 생성 순으로 정렬한 다음 최대 3개의 주문을 반환합니다.
    return f"{status} 상태인 {user} 사용자의 최근 주문 3건!"

tools = [get_recent_orders_by_status]

runnable_with_tools = prompt | model.bind_tools(tools)

with_message_history = RunnableWithMessageHistory(
    runnable_with_tools,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [ ]:
with_message_history.invoke(
    {"input": "안녕. 내 이름은 '나들'이라고 해. 너는 어떤 로봇이니?"},
    config={"configurable": {"session_id": "test_240510"}},
)

Parent run 095eee49-c17a-4e57-9f75-b94884a864c6 not found for run adf5da3c-6928-4c17-bcaa-f1bab22b9080. Treating as a root run.


AIMessage(content='안녕하세요, 나들님. 저는 주문봇으로 주문과 관련된 정보를 제공해드리는 로봇이에요. 어떤 도움이 필요하신가요?', response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 164, 'total_tokens': 225}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a26bd86b-1d88-4396-a6ba-c077e9ccb25e-0')

In [ ]:
response = with_message_history.invoke(
    {"input": "내 이름이 뭐라고 그랬지?"},
    config={"configurable": {"session_id": "test_240510"}},
    )
response

Parent run 777cb8b7-880c-422d-9dbd-139ab47f56f1 not found for run d5e5c8ee-010a-453a-b030-58067276b16d. Treating as a root run.


AIMessage(content="네 이름은 '나들'이라고 하셨어요. 어떤 도움이 필요하신가요?", response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 244, 'total_tokens': 279}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c6c6074d-02f8-4f62-96af-d897db3bdca8-0')

In [ ]:
response.response_metadata

{'token_usage': {'completion_tokens': 35,
  'prompt_tokens': 244,
  'total_tokens': 279},
 'model_name': 'gpt-3.5-turbo',
 'system_fingerprint': None,
 'finish_reason': 'stop',
 'logprobs': None}

In [ ]:
response = with_message_history.invoke(
    {"input": "내 아이디 역시 '나들'이야. '주문 완료' 상태인 주문들을 모두 알고 싶어"},
    config={"configurable": {"session_id": "test_240510"}},
)
response 

Parent run 5bde4fcb-4ba1-4a80-9a06-b1cb27ea910a not found for run c83a27f6-fe65-47ea-afa6-37221486622b. Treating as a root run.


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_eplFbDZUfIc5ccWFIUirSDsM', 'function': {'arguments': '{"user":"나들","status":"주문 완료"}', 'name': 'get_recent_orders_by_status'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 323, 'total_tokens': 349}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-85af0d1e-151e-488f-83c6-ff9e2817d363-0', tool_calls=[{'name': 'get_recent_orders_by_status', 'args': {'user': '나들', 'status': '주문 완료'}, 'id': 'call_eplFbDZUfIc5ccWFIUirSDsM'}])

In [ ]:
response.additional_kwargs

{'tool_calls': [{'id': 'call_eplFbDZUfIc5ccWFIUirSDsM',
   'function': {'arguments': '{"user":"나들","status":"주문 완료"}',
    'name': 'get_recent_orders_by_status'},
   'type': 'function'}]}

In [ ]:
# 복붙한다고 어제와 다른 session_id 수정하지 않아서 문제 발생한 듯
response = with_message_history.invoke(
    {"input": "'배송 완료' 상태인 주문들도 알려줘"},
    config={"configurable": {"session_id": "test2"}},
    )
response 

Parent run a4102aed-3b45-41d4-be3b-af3baf6aa9e2 not found for run f246d89a-b64d-4d96-a8d5-d3262c6da38b. Treating as a root run.


BadRequestError: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_ukKAHvQj3hOn0zfkvh1eaVMx", 'type': 'invalid_request_error', 'param': 'messages.[3].role', 'code': None}}

In [ ]:
response = with_message_history.invoke(
    {"input": "'배송 완료' 상태인 주문들도 알려줘"},
    config={"configurable": {"session_id": "test_240510"}},
    )
response 

Parent run 4896a516-aca7-4717-88af-9ee74f0b5fdd not found for run 851deb5a-ec0d-4a92-81f6-ac652b1e1f9a. Treating as a root run.


BadRequestError: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_eplFbDZUfIc5ccWFIUirSDsM", 'type': 'invalid_request_error', 'param': 'messages.[7].role', 'code': None}}

In [ ]:
from typing import Any

from langchain_core.runnables import Runnable, RunnableConfig


def try_except_tool(tool_args: dict, config: RunnableConfig) -> Runnable:
    try:
        get_recent_orders_by_status.invoke(tool_args, config=config)
    except Exception as e:
        return f"Calling tool with arguments:\n\n{tool_args}\n\nraised the following error:\n\n{type(e)}: {e}"


chain = with_message_history | (lambda msg: msg.tool_calls[0]["args"]) | try_except_tool
chain.invoke(
    "use get_recent_orders_by_status tool. the args are 'nadle', '배송 완료'. "
)

ValueError: Missing keys ['session_id'] in config['configurable'] Expected keys are ['session_id'].When using via .invoke() or .stream(), pass in a config; e.g., chain.invoke({'input': 'foo'}, {'configurable': {'session_id': '[your-value-here]'}})

In [ ]:
invoke(
    {"input": "'배송 완료' 상태인 주문들도 알려줘"},
    config={"configurable": {"session_id": "test_240510"}},
    )

In [ ]:
from typing import Any

from langchain_core.runnables import Runnable, RunnableConfig


def try_except_tool(tool_args: dict, config: RunnableConfig) -> Runnable:
    try:
        get_recent_orders_by_status.invoke(tool_args, config=config)
    except Exception as e:
        return f"Calling tool with arguments:\n\n{tool_args}\n\nraised the following error:\n\n{type(e)}: {e}"


chain = with_message_history | (lambda msg: msg.tool_calls[0]["args"]) | try_except_tool
chain.invoke(
    {"input": "use get_recent_orders_by_status tool. the args are 'nadle', '배송 완료'. "},
    config={"configurable": {"session_id": "test_240510"}},
)

Parent run db3ca572-750f-4927-b1dd-edeca6e620f9 not found for run 61673128-0ece-4f99-8f21-777c2339c194. Treating as a root run.


BadRequestError: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_eplFbDZUfIc5ccWFIUirSDsM", 'type': 'invalid_request_error', 'param': 'messages.[7].role', 'code': None}}

In [ ]:
# 바로 직전 메시지에 tool_calls 포함돼 있어 다음 메시지에 일반 메시지 입력 시 오류 발생하는 듯?

store["test_240510"].messages

[HumanMessage(content="안녕. 내 이름은 '나들'이라고 해. 너는 어떤 로봇이니?"),
 AIMessage(content='안녕하세요, 나들님. 저는 주문봇으로 주문과 관련된 정보를 제공해드리는 로봇이에요. 어떤 도움이 필요하신가요?', response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 164, 'total_tokens': 225}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a26bd86b-1d88-4396-a6ba-c077e9ccb25e-0'),
 HumanMessage(content='내 이름이 뭐라고 그랬지?'),
 AIMessage(content="네 이름은 '나들'이라고 하셨어요. 어떤 도움이 필요하신가요?", response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 244, 'total_tokens': 279}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c6c6074d-02f8-4f62-96af-d897db3bdca8-0'),
 HumanMessage(content="내 아이디 역시 '나들'이야. '주문 완료' 상태인 주문들을 모두 알고 싶어"),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_eplFbDZUfIc5ccWFIUirSDsM', 'function': {'arguments': '{"user":"나들","status":"주문 완료"}', 'name': 'get_recent

In [ ]:
store["test_240510"].messages[-1]

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_eplFbDZUfIc5ccWFIUirSDsM', 'function': {'arguments': '{"user":"나들","status":"주문 완료"}', 'name': 'get_recent_orders_by_status'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 323, 'total_tokens': 349}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-85af0d1e-151e-488f-83c6-ff9e2817d363-0', tool_calls=[{'name': 'get_recent_orders_by_status', 'args': {'user': '나들', 'status': '주문 완료'}, 'id': 'call_eplFbDZUfIc5ccWFIUirSDsM'}])

In [ ]:
from langchain_core.messages import AIMessage

def call_tools(msg: AIMessage) -> Runnable:
    """Simple sequential tool calling message. """
    tool_map = {tool.name: tool for tool in tools}
    tool_calls = msg.tool_calls.copy()
    for tool_call in tool_calls:
        tool_call["output"] = tool_map[tool_call["name"]].invoke(tool_call["args"])
    return tool_calls

In [ ]:
call_tools(store["test_240510"].messages[-1])

[{'name': 'get_recent_orders_by_status',
  'args': {'user': '나들', 'status': '주문 완료'},
  'id': 'call_eplFbDZUfIc5ccWFIUirSDsM',
  'output': '주문 완료 상태인 나들 사용자의 최근 주문 3건!'}]

In [ ]:
with_message_history.invoke(
    {"input": "안녕. 내 이름은 '나들'이라고 해. 너는 어떤 로봇이니?"},
    config={"configurable": {"session_id": "test_240510-1"}},
)

Parent run fb00032b-aab2-4b38-a09e-6fcbe2f4f5c5 not found for run becb41c6-1909-41d2-bafc-9424b7db4b2e. Treating as a root run.


AIMessage(content='안녕, 나들님! 저는 주문봇이에요. 주문과 관련된 정보를 제공하고 도와드릴 수 있어요. 궁금한 점이 있으면 언제든지 물어보세요! 😊\n안녕, 나들님! 주문과 관련된 정보를 제공하고 도와드릴 수 있는 주문봇이에요. 궁금한 점이 있으면 언제든지 물어보세요! 😊', response_metadata={'token_usage': {'completion_tokens': 138, 'prompt_tokens': 164, 'total_tokens': 302}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fb7f455b-0e3a-4284-b0f6-81ba103b819d-0')

In [ ]:
response = with_message_history.invoke(
    {"input": "do you remember my name?"},
    config={"configurable": {"session_id": "test_240510-1"}},
)
response

Parent run 45cd0f6e-f17f-4c69-8d11-3d65c02b076f not found for run 893a7ab6-eff9-4f83-ba95-9b2d0686a5a1. Treating as a root run.


AIMessage(content="Yes, your name is '나들'! How can I assist you today?", response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 315, 'total_tokens': 333}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1d0119f2-1483-4b37-bba3-ccaaa031b444-0')

In [ ]:
bool(response.additional_kwargs)

False

In [ ]:
from langchain_core.messages import AIMessage

def determine_tool_usage(msg: AIMessage) -> Runnable:
    """Determine whether to use the tool based on the given condition."""
    if msg.additional_kwargs: 
        tool_map = {tool.name: tool for tool in tools}
        tool_calls = msg.tool_calls.copy()
        for tool_call in tool_calls:
            tool_call["output"] = tool_map[tool_call["name"]].invoke(tool_call["args"])
        return tool_call
    else:
        return msg.content

In [ ]:
chain_with_history_n_tools = with_message_history | determine_tool_usage
chain_with_history_n_tools

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  chat_history: RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
}), config={'run_name': 'insert_history'})
| RunnableBinding(bound=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='너는 뛰어나고 유능한 주문봇이야이전 대화와 현재 고객이 입력한 메시지 모두를 꼼꼼히 파악하여 답변해줘.')), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
  | RunnableBinding(bound=ChatOpenAI(clien

In [ ]:
response = chain_with_history_n_tools.invoke(
    {"input": "안녕. 내 이름은 '나들'이라고 해. 너는 어떤 로봇이니?"},
    config={"configurable": {"session_id": "test_240510-2"}},
)
response

Parent run 2135a47d-27ce-4b1c-a2ee-be663df39354 not found for run 8d2a2e0a-d5e6-4298-acad-25b902dad2d6. Treating as a root run.


'안녕, 나들님! 저는 주문봇이라고 해요. 주문과 관련된 정보를 제공하고 도와드릴 수 있는봇이에요. 무엇을 도와드릴까요?'

In [ ]:
response = chain_with_history_n_tools.invoke(
    {"input": "내 아이디 역시 '나들'이야. '주문 완료' 상태인 주문들을 모두 알고 싶어"},
    config={"configurable": {"session_id": "test_240510-2"}},
)
response

Parent run 13645d02-e319-4cb8-8874-5484bc549ffd not found for run fcfdaccc-c7bd-4284-90cd-201270ac18c3. Treating as a root run.


{'name': 'get_recent_orders_by_status',
 'args': {'user': '나들', 'status': '주문 완료'},
 'id': 'call_SAQXsob4o7HHJV7To2JyuXzW',
 'output': '주문 완료 상태인 나들 사용자의 최근 주문 3건!'}

In [ ]:
response = chain_with_history_n_tools.invoke(
    {"input": "내 이름 기억해?"},
    config={"configurable": {"session_id": "test_240510-2"}},
)
response

Parent run b61c98b2-ad8b-4089-8c79-bc3f42e83551 not found for run eacf09f0-eec7-4ff8-b8f3-51968d75319a. Treating as a root run.


BadRequestError: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_SAQXsob4o7HHJV7To2JyuXzW", 'type': 'invalid_request_error', 'param': 'messages.[5].role', 'code': None}}

In [ ]:
response = chain_with_history_n_tools.invoke(
    {"input": "'배송 완료' 상태인 주문들도 모두 알고 싶어"},
    config={"configurable": {"session_id": "test_240510-2"}},
)
response

Parent run d94c8ee5-c409-4561-9a05-c59626eeb1fd not found for run 66ed36fc-f759-4665-947a-c802a1858587. Treating as a root run.


BadRequestError: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_SAQXsob4o7HHJV7To2JyuXzW", 'type': 'invalid_request_error', 'param': 'messages.[5].role', 'code': None}}

In [ ]:
store["test_240510-2"]

InMemoryChatMessageHistory(messages=[HumanMessage(content="안녕. 내 이름은 '나들'이라고 해. 너는 어떤 로봇이니?"), AIMessage(content='안녕, 나들님! 저는 주문봇이라고 해요. 주문과 관련된 정보를 제공하고 도와드릴 수 있는봇이에요. 무엇을 도와드릴까요?', response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 164, 'total_tokens': 232}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-65513e53-1215-4cca-a630-ec0dd2527cfe-0'), HumanMessage(content="내 아이디 역시 '나들'이야. '주문 완료' 상태인 주문들을 모두 알고 싶어"), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_SAQXsob4o7HHJV7To2JyuXzW', 'function': {'arguments': '{"user":"나들","status":"주문 완료"}', 'name': 'get_recent_orders_by_status'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 276, 'total_tokens': 302}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f07ec416-eb4f-4725-ad4c-d887065a5d26-0', tool_calls=

In [ ]:
store["test_240510-2"].messages

[HumanMessage(content="안녕. 내 이름은 '나들'이라고 해. 너는 어떤 로봇이니?"),
 AIMessage(content='안녕, 나들님! 저는 주문봇이라고 해요. 주문과 관련된 정보를 제공하고 도와드릴 수 있는봇이에요. 무엇을 도와드릴까요?', response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 164, 'total_tokens': 232}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-65513e53-1215-4cca-a630-ec0dd2527cfe-0'),
 HumanMessage(content="내 아이디 역시 '나들'이야. '주문 완료' 상태인 주문들을 모두 알고 싶어"),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_SAQXsob4o7HHJV7To2JyuXzW', 'function': {'arguments': '{"user":"나들","status":"주문 완료"}', 'name': 'get_recent_orders_by_status'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 276, 'total_tokens': 302}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f07ec416-eb4f-4725-ad4c-d887065a5d26-0', tool_calls=[{'name': 'get_recent_orders_by_s

In [ ]:
response

{'name': 'get_recent_orders_by_status',
 'args': {'user': '나들', 'status': '주문 완료'},
 'id': 'call_SAQXsob4o7HHJV7To2JyuXzW',
 'output': '주문 완료 상태인 나들 사용자의 최근 주문 3건!'}

In [ ]:
from operator import itemgetter

itemgetter("output")

operator.itemgetter('output')

In [ ]:
itemgetter("output")(response)

'주문 완료 상태인 나들 사용자의 최근 주문 3건!'

In [ ]:
store["test_240510-2"].messages

[HumanMessage(content="안녕. 내 이름은 '나들'이라고 해. 너는 어떤 로봇이니?"),
 AIMessage(content='안녕, 나들님! 저는 주문봇이라고 해요. 주문과 관련된 정보를 제공하고 도와드릴 수 있는봇이에요. 무엇을 도와드릴까요?', response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 164, 'total_tokens': 232}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-65513e53-1215-4cca-a630-ec0dd2527cfe-0'),
 HumanMessage(content="내 아이디 역시 '나들'이야. '주문 완료' 상태인 주문들을 모두 알고 싶어"),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_SAQXsob4o7HHJV7To2JyuXzW', 'function': {'arguments': '{"user":"나들","status":"주문 완료"}', 'name': 'get_recent_orders_by_status'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 276, 'total_tokens': 302}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f07ec416-eb4f-4725-ad4c-d887065a5d26-0', tool_calls=[{'name': 'get_recent_orders_by_s

In [ ]:
from langchain_core.messages import AIMessage

store["test_240510-2"].messages.append(AIMessage(content=itemgetter("output")(response)))

In [ ]:
store["test_240510-2"].messages

[HumanMessage(content="안녕. 내 이름은 '나들'이라고 해. 너는 어떤 로봇이니?"),
 AIMessage(content='안녕, 나들님! 저는 주문봇이라고 해요. 주문과 관련된 정보를 제공하고 도와드릴 수 있는봇이에요. 무엇을 도와드릴까요?', response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 164, 'total_tokens': 232}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-65513e53-1215-4cca-a630-ec0dd2527cfe-0'),
 HumanMessage(content="내 아이디 역시 '나들'이야. '주문 완료' 상태인 주문들을 모두 알고 싶어"),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_SAQXsob4o7HHJV7To2JyuXzW', 'function': {'arguments': '{"user":"나들","status":"주문 완료"}', 'name': 'get_recent_orders_by_status'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 276, 'total_tokens': 302}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f07ec416-eb4f-4725-ad4c-d887065a5d26-0', tool_calls=[{'name': 'get_recent_orders_by_s

In [ ]:
response = chain_with_history_n_tools.invoke(
    {"input": "'배송 완료' 상태인 주문들도 모두 알고 싶어"},
    config={"configurable": {"session_id": "test_240510-2"}},
)
response

Parent run 85d95484-53a4-4509-a8ce-918b4d8935f8 not found for run 317520dc-9b4f-41cc-b8cb-fc0be80448ba. Treating as a root run.


BadRequestError: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_SAQXsob4o7HHJV7To2JyuXzW", 'type': 'invalid_request_error', 'param': 'messages.[5].role', 'code': None}}

In [ ]:
store["test_240510-2"].messages

[HumanMessage(content="안녕. 내 이름은 '나들'이라고 해. 너는 어떤 로봇이니?"),
 AIMessage(content='안녕, 나들님! 저는 주문봇이라고 해요. 주문과 관련된 정보를 제공하고 도와드릴 수 있는봇이에요. 무엇을 도와드릴까요?', response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 164, 'total_tokens': 232}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-65513e53-1215-4cca-a630-ec0dd2527cfe-0'),
 HumanMessage(content="내 아이디 역시 '나들'이야. '주문 완료' 상태인 주문들을 모두 알고 싶어"),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_SAQXsob4o7HHJV7To2JyuXzW', 'function': {'arguments': '{"user":"나들","status":"주문 완료"}', 'name': 'get_recent_orders_by_status'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 276, 'total_tokens': 302}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f07ec416-eb4f-4725-ad4c-d887065a5d26-0', tool_calls=[{'name': 'get_recent_orders_by_s

---

In [ ]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [ ]:
model = ChatOpenAI()

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

@tool
def get_recent_orders_by_status(user, status):
    """
    Retrieve the three most recent orders with a specific status for the given user.
    """
    # status_changes 를 통해 필터링하고, 주문 생성 순으로 정렬한 다음 최대 3개의 주문을 반환합니다.
    return f"{status} 상태인 {user} 사용자의 최근 주문 3건!"

tools = [get_recent_orders_by_status]

runnable_with_tools = prompt | model.bind_tools(tools)

with_message_history = RunnableWithMessageHistory(
    runnable_with_tools,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

from langchain_core.messages import AIMessage

def determine_tool_usage(msg: AIMessage) -> Runnable:
    """Determine whether to use the tool based on the given condition."""
    if msg.additional_kwargs: 
        tool_map = {tool.name: tool for tool in tools}
        tool_calls = msg.tool_calls.copy()
        for tool_call in tool_calls:
            tool_call["output"] = tool_map[tool_call["name"]].invoke(tool_call["args"])
        return tool_call
    else:
        return msg.content
    
chain_with_history_n_tools = with_message_history | determine_tool_usage

In [ ]:
response = chain_with_history_n_tools.invoke(
    {"input": "안녕, 내 이름은 '나들'이라고 해"},
    config={"configurable": {"session_id": "test_240510-3"}},
)
response

Parent run 9a81b6b3-9e40-4e66-a80b-9d7c90f6dda1 not found for run 8bf795c1-1f9d-4a5c-bb4b-86f9ef435e5e. Treating as a root run.


KeyError: "Input to ChatPromptTemplate is missing variables {'agent_scratchpad'}.  Expected: ['agent_scratchpad', 'input'] Received: ['input', 'chat_history']"

---

In [ ]:
@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int

@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent

In [ ]:
from langchain.agents import initialize_agent
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory

# Initialize the tools
tools = [multiply, add, exponentiate]

# Initialize the LLM
llm = OpenAI(temperature=0)

# Initialize memory
memory = ConversationBufferMemory(memory_key="chat_history")

# Initialize the agent with memory
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True, memory=memory)

# Run the agent
query = "What's 23 times 7"
result = agent.run(query)
print(result)

ValueError: ZeroShotAgent does not support multi-input tool multiply.

In [ ]:

# Run the agent again with a follow-up query
follow_up_query = "What river does Paris lie on?"
result = agent.run(follow_up_query)
print(result)

---

In [ ]:
from langchain.tools.render import render_text_description

tools = [get_recent_orders_by_status]
rendered_tools = render_text_description(tools)

system_prompt = f"""You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments' keys."""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

chain = prompt | model

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [ ]:
response = with_message_history.invoke(
    {"input": "안녕, 내 이름은 '나들'이라고 해"},
    config={"configurable": {"session_id": "test_240510-4"}},
)
response

Parent run 7b80c389-ec7e-4f00-82a1-3506bd4d691f not found for run d10b928d-c23c-4bbd-970f-41cda4ebedf5. Treating as a root run.


AIMessage(content='{\n    "name": "get_recent_orders_by_status",\n    "arguments": ["나들", "status"]\n}', response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 115, 'total_tokens': 139}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c37c0a64-c85c-43bb-bdc9-00aa4d8affb0-0')

In [ ]:
store["test_240510-4"].messages

[HumanMessage(content="안녕, 내 이름은 '나들'이라고 해"),
 AIMessage(content='{\n    "name": "get_recent_orders_by_status",\n    "arguments": ["나들", "status"]\n}', response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 115, 'total_tokens': 139}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c37c0a64-c85c-43bb-bdc9-00aa4d8affb0-0')]

---

In [ ]:
from langchain import hub
from langchain.memory import ChatMessageHistory

prompt = hub.pull("hwchase17/react")

memory = ChatMessageHistory(session_id="test_240510-5")

In [ ]:
@tool
def get_recent_orders_by_status(user, status):
    """
    Retrieve the three most recent orders with a specific status for the given user.
    """
    # status_changes 를 통해 필터링하고, 주문 생성 순으로 정렬한 다음 최대 3개의 주문을 반환합니다.
    return f"{status} 상태인 {user} 사용자의 최근 주문 3건!"

tools = [get_recent_orders_by_status]

In [ ]:
from langchain_openai import OpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.runnables.history import RunnableWithMessageHistory

llm = OpenAI(temperature=0)
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True)

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [ ]:
agent_with_chat_history.invoke(
    {"input": "내 이름은 '나들'이야"},
    config={"configurable": {"session_id": "test_240510-5"}},
)

Parent run 0cb00327-1bb8-42eb-9281-9904d5a0ec8d not found for run 2da1445a-9e7b-4d9a-93b2-ef60b9da3f40. Treating as a root run.


ValidationError: 1 validation error for get_recent_orders_by_statusSchema
status
  field required (type=value_error.missing)

---

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.tools import tool

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "너는 뛰어나고 유능한 주문봇이야"
            "이전 대화와 현재 고객이 입력한 메시지 모두를 꼼꼼히 파악하여 답변해줘."
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),

    ]
)

model = ChatOpenAI()

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

@tool
def get_recent_orders_by_status(user, status):
    """
    Retrieve the three most recent orders with a specific status for the given user.
    """
    # status_changes 를 통해 필터링하고, 주문 생성 순으로 정렬한 다음 최대 3개의 주문을 반환합니다.
    return f"{status} 상태인 {user} 사용자의 최근 주문 3건!"

tools = [get_recent_orders_by_status]

def determine_tool_usage(msg: AIMessage) -> Runnable:
    """Determine whether to use the tool based on the given condition."""
    if msg.additional_kwargs: 
        tool_map = {tool.name: tool for tool in tools}
        tool_calls = msg.tool_calls.copy()
        for tool_call in tool_calls:
            tool_call["output"] = tool_map[tool_call["name"]].invoke(tool_call["args"])
        return tool_call
    else:
        return msg.content

runnable_with_tools = prompt | model.bind_tools(tools) | determine_tool_usage

with_message_history = RunnableWithMessageHistory(
    runnable_with_tools,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [ ]:
response = with_message_history.invoke(
    {"input": "내 이름은 '나들'이야"},
    config={"configurable": {"session_id": "test_240510-4"}},
)
response

Parent run 28975645-3eb6-481f-a208-af655b6f1e3e not found for run c1a212b9-fb58-4db8-984a-bcf958591d80. Treating as a root run.


'반갑습니다, 나들님! 무엇을 도와드릴까요?'

In [ ]:
store["test_240510-4"].messages

[HumanMessage(content="내 이름은 '나들'이야"),
 AIMessage(content='반갑습니다, 나들님! 무엇을 도와드릴까요?')]

In [ ]:
response = with_message_history.invoke(
    {"input": "내 아이디도 '나들'이거든. '주문 완료' 상태인 모든 내 주문 내역을 알고 싶어"},
    config={"configurable": {"session_id": "test_240510-4"}},
)
response

Parent run 6124ca70-707d-43a3-b12c-67b54ddaace2 not found for run 69870ff7-af20-4e3b-a4e0-0b3e3905cdce. Treating as a root run.


{'name': 'get_recent_orders_by_status',
 'args': {'user': '나들', 'status': '주문 완료'},
 'id': 'call_6gJ0ZZsivH1fLkV6v9gHR5n7',
 'output': '주문 완료 상태인 나들 사용자의 최근 주문 3건!'}

In [ ]:
response = with_message_history.invoke(
    {"input": "내 이름 기억해?"},
    config={"configurable": {"session_id": "test_240510-4"}},
)
response

Parent run 669f0af2-6cd4-4044-a06c-dcf98e35b2c2 not found for run 8a36f64f-8057-43f6-a4d6-fce7167e4a32. Treating as a root run.


{'name': 'get_recent_orders_by_status',
 'args': {'user': '나들', 'status': '주문 완료'},
 'id': 'call_PnXLnsVhLU07neWUL6PQFIfA',
 'output': '주문 완료 상태인 나들 사용자의 최근 주문 3건!'}

In [ ]:
response = with_message_history.invoke(
    {"input": "아니, 내 이름 기억하냐고~~"},
    config={"configurable": {"session_id": "test_240510-4"}},
)
response

Parent run 8b177ac5-e378-4adc-b76f-222a5f65c367 not found for run 26a9a1f5-8ecc-42db-848a-74617f471a71. Treating as a root run.


{'name': 'get_recent_orders_by_status',
 'args': {'user': '나들', 'status': '주문 완료'},
 'id': 'call_nZBKg3h7ex7o8xubCQXx6BTv',
 'output': '주문 완료 상태인 나들 사용자의 최근 주문 3건!'}

In [ ]:
response = with_message_history.invoke(
    {"input": "내 이름은?"},
    config={"configurable": {"session_id": "test_240510-4"}},
)
response

Parent run 7dfd2d4c-1ece-46c5-9eb6-8f1f812260cf not found for run fddadd65-9f9e-4612-85a0-7642b579aedd. Treating as a root run.


{'name': 'get_recent_orders_by_status',
 'args': {'user': '나들', 'status': '주문 완료'},
 'id': 'call_Y3bW9sTOUjr6e2jBoP0PBIhu',
 'output': '주문 완료 상태인 나들 사용자의 최근 주문 3건!'}

---

In [ ]:
chain = prompt | model.bind_tools(tools)

In [ ]:
response = chain.invoke({"input": "나는 '나들'이라고 해",
                         "chat_history": []}
                         )
response

AIMessage(content="안녕하세요, '나들'님! 무엇을 도와드릴까요?", response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 142, 'total_tokens': 171}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ff2b5ee4-4f48-4389-ac15-f6c696ab961f-0')

In [ ]:
history = ''
history += ' ' + response.content
history

" 안녕하세요, '나들'님! 무엇을 도와드릴까요?"

In [ ]:
history = history.lstrip()
history

"안녕하세요, '나들'님! 무엇을 도와드릴까요?"

In [ ]:
response = chain.invoke({"input": "내 아이디도 '나들'이거든. '주문 완료' 상태인 모든 내 주문 내역을 알고 싶어",
                         "chat_history": [history]}
                         )
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_wMceT7F8NdEdDRUomZDJP9MF', 'function': {'arguments': '{"user":"나들","status":"주문 완료"}', 'name': 'get_recent_orders_by_status'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 202, 'total_tokens': 228}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ed8145e1-dea5-47e1-bec8-bb49eb1daae7-0', tool_calls=[{'name': 'get_recent_orders_by_status', 'args': {'user': '나들', 'status': '주문 완료'}, 'id': 'call_wMceT7F8NdEdDRUomZDJP9MF'}])

---

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.tools import tool

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "너는 뛰어나고 유능한 주문봇이야"
            "이전 대화와 현재 고객이 입력한 메시지 모두를 꼼꼼히 파악하여 답변해줘."
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),

    ]
)

model = ChatOpenAI()

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

@tool
def get_recent_orders_by_status(user, status):
    """
    Retrieve the three most recent orders with a specific status for the given user.
    """
    # status_changes 를 통해 필터링하고, 주문 생성 순으로 정렬한 다음 최대 3개의 주문을 반환합니다.
    return f"{status} 상태인 {user} 사용자의 최근 주문 3건!"

tools = [get_recent_orders_by_status]

def determine_tool_usage(msg: AIMessage) -> Runnable:
    """Determine whether to use the tool based on the given condition."""
    if msg.additional_kwargs: 
        tool_map = {tool.name: tool for tool in tools}
        tool_calls = msg.tool_calls.copy()
        for tool_call in tool_calls:
            tool_call["output"] = tool_map[tool_call["name"]].invoke(tool_call["args"])
        return tool_call["output"]
    else:
        return msg.content

runnable_with_tools = prompt | model.bind_tools(tools) | determine_tool_usage

with_message_history = RunnableWithMessageHistory(
    runnable_with_tools,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [ ]:
response = with_message_history.invoke(
    {"input": "내 이름은 '나들'이야"},
    config={"configurable": {"session_id": "test_240510-5"}},
)
response

Parent run ba1b8042-6358-4cc0-92bf-4a330549746a not found for run 05cffd97-ab49-47e4-bf49-9775a64cef80. Treating as a root run.


'반가워, 나들님! 무엇을 도와드릴까요?'

In [ ]:
store["test_240510-5"].messages

[HumanMessage(content="내 이름은 '나들'이야"),
 AIMessage(content='반가워, 나들님! 무엇을 도와드릴까요?')]

In [ ]:
response = with_message_history.invoke(
    {"input": "내 아이디도 '나들'이거든. '주문 완료' 상태인 모든 내 주문 내역을 알고 싶어"},
    config={"configurable": {"session_id": "test_240510-5"}},
)
response

Parent run 9913f2dc-dac2-4e52-b972-dfd3ddf729cd not found for run c8690690-431f-4a77-9af3-02d3c43dcd9d. Treating as a root run.


'주문 완료 상태인 나들 사용자의 최근 주문 3건!'

In [ ]:
response = with_message_history.invoke(
    {"input": "내 이름 기억해?"},
    config={"configurable": {"session_id": "test_240510-5"}},
)
response

Parent run 47745a2d-8c82-4b18-81c0-5e69ccbee9df not found for run 46f8792c-09b9-4be2-aaab-5faec62a1510. Treating as a root run.


'주문 완료 상태인 나들 사용자의 최근 주문 3건!'

In [ ]:
response = with_message_history.invoke(
    {"input": "내 이름이 뭔데?"},
    config={"configurable": {"session_id": "test_240510-5"}},
)
response

Parent run d7b77cf3-275d-4e9d-9967-444e4f4af7b6 not found for run 6f04ca2a-1afa-45c8-ab3c-56ad2c166033. Treating as a root run.


"네 이름은 '나들'이에요! 주문 완료 상태인 주문 내역을 확인해볼까요?"

In [ ]:
response = with_message_history.invoke(
    {"input": "아니 그냥 가위바위보 게임이나 하자"},
    config={"configurable": {"session_id": "test_240510-5"}},
)
response

Parent run 150f14e9-de46-4f1b-89b8-f1859a64690d not found for run 46b94728-67e6-40db-ad87-dc2d85ad756d. Treating as a root run.


'가위바위보 게임을 시작합니다! 가위, 바위, 보 중에서 선택해주세요.'

In [ ]:
response = with_message_history.invoke(
    {"input": "아 아니다. '배송 완료'인 주문 내역부터 모두 확인해줘"},
    config={"configurable": {"session_id": "test_240510-5"}},
)
response

Parent run 1f3a05e8-c592-4aca-bc2f-b654b1d74929 not found for run e310585c-1e48-4bbb-899b-621f676fc0a2. Treating as a root run.


'배송 완료 상태인 나들 사용자의 최근 주문 3건!'

In [ ]:
response = with_message_history.invoke(
    {"input": "다시 가위바위보 하자"},
    config={"configurable": {"session_id": "test_240510-5"}},
)
response

Parent run 7e1ee1ea-c4f6-4165-a690-649ef63fd217 not found for run 701d4e2b-2f5e-49fd-a535-191af1fd0d05. Treating as a root run.


'가위바위보 게임을 시작합니다! 가위, 바위, 보 중에서 선택해주세요.'

In [ ]:
response = with_message_history.invoke(
    {"input": "각자 뭐를 낼 지 다음 차례에 동시에 외치기로 해"},
    config={"configurable": {"session_id": "test_240510-5"}},
)
response

Parent run 0281445d-54a9-4652-96e1-a38d9b5e096d not found for run 56b3634e-cccc-4745-ab18-c035cf1e0c8f. Treating as a root run.


'알겠어요! 다음 차례에 동시에 가위, 바위, 보 중에서 선택하겠습니다. 준비가 되면 알려주세요!'

In [ ]:
response = with_message_history.invoke(
    {"input": "가위"},
    config={"configurable": {"session_id": "test_240510-5"}},
)
response

Parent run 4470a481-5d22-4084-9399-c39e089b8c6d not found for run e10812c6-fb4f-45c3-aece-e5eb880ce34c. Treating as a root run.


'바위'

In [ ]:
response = with_message_history.invoke(
    {"input": "누가 이긴 거야?"},
    config={"configurable": {"session_id": "test_240510-5"}},
)
response

Parent run c31a29be-9e5f-40a6-92d2-e0e538e1d4cb not found for run 1a4f680f-d99a-4b41-ab20-50b6e08ebf77. Treating as a root run.


"너의 선택은 '가위'이고, 나는 '바위'를 냈어. 그래서 나의 승리야! 다시 한 판 하거나 다른 게임을 하고 싶다면 언제든지 말해줘."

In [ ]:
store["test_240510-5"].messages

[HumanMessage(content="내 이름은 '나들'이야"),
 AIMessage(content='반가워, 나들님! 무엇을 도와드릴까요?'),
 HumanMessage(content="내 아이디도 '나들'이거든. '주문 완료' 상태인 모든 내 주문 내역을 알고 싶어"),
 AIMessage(content='주문 완료 상태인 나들 사용자의 최근 주문 3건!'),
 HumanMessage(content='내 이름 기억해?'),
 AIMessage(content='주문 완료 상태인 나들 사용자의 최근 주문 3건!'),
 HumanMessage(content='내 이름이 뭔데?'),
 AIMessage(content="네 이름은 '나들'이에요! 주문 완료 상태인 주문 내역을 확인해볼까요?"),
 HumanMessage(content='아니 그냥 가위바위보 게임이나 하자'),
 AIMessage(content='가위바위보 게임을 시작합니다! 가위, 바위, 보 중에서 선택해주세요.'),
 HumanMessage(content="아 아니다. '배송 완료'인 주문 내역부터 모두 확인해줘"),
 AIMessage(content='배송 완료 상태인 나들 사용자의 최근 주문 3건!'),
 HumanMessage(content='다시 가위바위보 하자'),
 AIMessage(content='가위바위보 게임을 시작합니다! 가위, 바위, 보 중에서 선택해주세요.'),
 HumanMessage(content='각자 뭐를 낼 지 다음 차례에 동시에 외치기로 해'),
 AIMessage(content='알겠어요! 다음 차례에 동시에 가위, 바위, 보 중에서 선택하겠습니다. 준비가 되면 알려주세요!'),
 HumanMessage(content='가위'),
 AIMessage(content='바위'),
 HumanMessage(content='누가 이긴 거야?'),
 AIMessage(content="너의 선택은 '가위'이고, 나는 '바위'를 냈어. 그래서 나의 승리야! 다시 한 판 하